In [179]:
import pandas as pd

In [180]:
genes_set = pd.read_csv('gsets_1')
genes_set = genes_set.rename(columns={"YAL002W$YAL011W": "pairs"})
sl_labels = pd.read_csv('sl_1')
pairs_terms = genes_set.copy()
pairs_terms['sl'] = sl_labels
pairs_terms = pairs_terms.set_index('pairs')

In [181]:
from goatools.base import get_godag
godag = get_godag("go-basic.obo", optional_attrs='relationship')

  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2019-10-07) 47,285 GO Terms; optional_attrs(relationship)


In [182]:
from goatools.anno.gaf_reader import GafReader
ogaf = GafReader("goa_yeast.gaf")

HMS:0:00:04.747125  91,452 annotations READ: goa_yeast.gaf 


In [183]:
ogaf.associations[1]

ntgafobj(DB='UniProtKB', DB_ID='A0A023PYF4', DB_Symbol='YER145C-A', Qualifier=set(), GO_ID='GO:0005575', DB_Reference={'GO_REF:0000015'}, Evidence_Code='ND', With_From=set(), NS='CC', DB_Name={'Uncharacterized protein YER145C-A'}, DB_Synonym={'YER145C-A'}, DB_Type='protein', Taxon=[559292], Date=datetime.date(2003, 7, 30), Assigned_By='SGD', Extension=None, Gene_Product_Form_ID=set())

In [213]:
from collections import defaultdict
lookup_terms = defaultdict(set)
for assoc in ogaf.associations:
    for syn in assoc.DB_Synonym:
        lookup_terms[syn].add(assoc.GO_ID)

In [214]:
sum([1 for go_id in lookup_terms.values() for go_id in go_ids])

1255320

In [215]:
len(lookup_terms.keys())

17435

In [216]:
go_ids = set()
for v in lookup_terms.values():
    for go_id in v:
        go_ids.add(go_id)

In [217]:
from goatools.gosubdag.gosubdag import GoSubDag
optional_relationships = {'regulates', 'negatively_regulates', 'positively_regulates'}
gosubdag_r1 = GoSubDag(go_ids, godag, relationships=optional_relationships, prt=None)
parents = gosubdag_r1.rcntobj.go2parents

In [218]:
for k in lookup_terms.keys():
    parents_go_ids = set()
    for go_id in lookup_terms.get(k):
        parents_go_ids.update(parents.get(go_id))
    lookup_terms[k].update(parents_go_ids)

In [219]:
sum([1 for go_id in lookup_terms.values() for go_id in go_ids])

107103205

In [197]:
len(lookup_terms.keys())

17435

In [220]:
terms = defaultdict(set)
gene_pairs = pairs_terms.index.values
gens_terms = {}
for idx, pair in enumerate(gene_pairs):
#     if idx % 1000 == 0:
#         print("working on {}/{}".format(idx, len(gene_pairs)))
    for gene in pair.split("$"):
        go_ids = lookup_terms.get(gene)
        if go_ids is None:
            print("Error", gene)
        else:
            for go_id in go_ids:
                term = godag.get(go_id)
                if term.namespace == 'biological_process':
                    terms[go_id].add(pair)
 

Error YIL168W
Error YIL170W


In [221]:
pairs_terms = pd.concat([pd.DataFrame(columns=terms.keys()), pairs_terms], sort=False)
pairs_terms = pairs_terms.fillna(0)

In [ ]:
for term, pairs in terms.items():
    pairs_terms.loc[pairs, [term]] += 1

In [ ]:
pairs_terms.head()

In [ ]:
pairs_terms.describe()

In [ ]:
# remove column without no information
nunique = pairs_terms.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
print("dropping {} columns".format(len(cols_to_drop)))
pairs_terms.drop(cols_to_drop, axis=1)

In [ ]:
pairs_terms.to_pickle("pairs_terms.pkl")
# pairs_terms = pd.read_pickle("pairs_terms.pkl")

In [ ]:
X = pairs_terms.loc[:, pairs_terms.columns != 'sl']
Y = pairs_terms['sl']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs')

In [ ]:
model.fit(X, Y)

In [ ]:
standard_model = pd.read_csv('all.onto_genes.features.cv_all.rf.predictions')
standard_model = standard_model.rename(columns={"0": "prob"})
standard_model['sl'] = -1
standard_model.loc[standard_model['prob'] >= 0.5,'sl'] = 1
standard_model.loc[standard_model['prob'] < 0.5,'sl'] = 0

standard_model['pairs'] = pairs_terms.index.values
standard_model = standard_model.set_index('pairs')

In [ ]:
# standard_model.to_pickle("standard_model.pkl")
standard_model = pd.read_pickle("standard_model.pkl")

In [ ]:
y_pred = model.predict(X)
from sklearn.metrics import accuracy_score
accuracy_score(Y, standard_model['sl'])

In [ ]:
accuracy_score(Y, y_pred)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['non sl', 'sl']
print("theirs:")
print(classification_report(Y, standard_model['sl'], target_names=target_names))

In [ ]:
print("ours:")
print(classification_report(Y, y_pred, target_names=target_names))

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
# generate a no skill prediction (majority class)
ns_probs = [0 for _ in range(len(Y))]
# predict probabilities
lr_probs = model.predict_proba(X)
# # keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]
# calculate scores
ns_auc = roc_auc_score(Y, ns_probs)
lr_auc = roc_auc_score(Y, lr_probs)
standard_auc = roc_auc_score(Y, standard_model['prob'])
# summarize scores
print('Baseline: ROC AUC=%.3f' % (ns_auc))
print('Logistic: ROC AUC=%.3f' % (lr_auc))
print('Standard: ROC AUC=%.3f' % (standard_auc))
# calculate roc curves
ns_fpr, ns_tpr, _ = roc_curve(Y, ns_probs)
lr_fpr, lr_tpr, _ = roc_curve(Y, lr_probs)
base_fpr, base_tpr, _ = roc_curve(Y, standard_model['prob'])
# plot the roc curve for the model
pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='Baseline')
pyplot.plot(lr_fpr, lr_tpr, label='Logistic', color='green')
pyplot.plot(base_fpr, base_tpr, label='Standard', color='red')
# axis labels
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()